In [2]:

import os
import pika
import json
# from icecream.icecream import ic

if os.getcwd() != "C:/Users/Jerome/PycharmProjects/FASTgres":
    os.chdir("C:/Users/Jerome/PycharmProjects/FASTgres")

from fastgres.workloads.workload import Workload
from fastgres.baseline.database_connection import DatabaseConnection
from fastgres.async_components.node import SenderNode

# PG_STACK_OVERFLOW = "dbname=stack_overflow user=postgres password=postgres host=localhost port=5432"

class ProviderSender(SenderNode):

    def __init__(self, name: str, queues: list[str]):
        super().__init__(name, queues)

    def __str__(self):
        return f"Query Provider Node {self.name}"

workload = Workload("fastgres/workloads/queries/stack/", "stack")
workload.load_queries()

test_queries_names = workload.queries[:1000]
test_queries = [workload.read_query(_) for _ in test_queries_names]
query_dict = {str(test_queries_names[i]): str(test_queries[i]) for i in range(len(test_queries_names))}
message = {"message": "provider.dispatcher.query",
           "payload": query_dict}
# ic(message)
# database_connection = DatabaseConnection(PG_STACK_OVERFLOW, "stack_connection")


Loading Queries: 6191it [00:00, 55405.82it/s]


In [3]:
provider = ProviderSender("Query Provider", ["query_queue"])
try:
    provider.send_message(json.dumps(message), provider.queues[0])
    print("Sent message")
except Exception as e:
    print(f"Failed to send message: {e}")
    pass
finally:
    provider.close()
    # provider.msgThread.join()

# wl_node.channel.basic_publish(exchange='', routing_key=wl_node.queue, body="util.ping", properties=pika.BasicProperties(delivery_mode=2))
# wl_node.channel.basic_publish(exchange='', routing_key=wl_node.queue, body=f"query.{workload.queries[1]}", properties=pika.BasicProperties(delivery_mode=2))
# wl_node.close()

Sent message
